# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

~~[SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[ResNet34](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)~~
~~[PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)~~
~~[InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[BNInception](https://github.com/Cadene/pretrained-models.pytorch#bninception)~~
~~[InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)~~
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [ResNeXt101_64x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [SE-ResNeXt50_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet131](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)


 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc
from tqdm import tqdm

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import *
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
2,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
3,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
4,49850884a00703afe5aab78c3ce074d2d4acae30.jpg,../data/cleaned/none/49850884a00703afe5aab78c3...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 46/46 [00:09<00:00,  5.02it/s]


Image reading complete.
Image array shape: (299, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--lr", type=float, default=LEARNING_RATE)
    parser.add_argument("--weight_decay", type=float, default=WEIGHT_DECAY)
    parser.add_argument("--epsilon", type=float, default=EPSILON)
    parser.add_argument("--amsgrad", type=bool, default=AMSGRAD)
    parser.add_argument("--betas", default=BETAS)
    parser.add_argument("--eta_min", type=float, default=ETA_MIN)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = resnext101_32x4d()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.hparams.lr, 
            betas=self.hparams.betas, 
            eps=self.hparams.epsilon, 
            weight_decay=self.hparams.weight_decay,
            amsgrad=self.hparams.amsgrad
        )
        self.scheduler = WarmRestart(
            self.optimizer, 
            T_max=15, 
            T_mult=3, 
            eta_min=self.hparams.eta_min
        )
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=2)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())

        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()
torch.cuda.empty_cache()

log_notes = "patience/min_epochs/max_epochs decreased to 11/75/100, amsgrad changed to true"

# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Log parameters
logger.info(f"backbone: {hparams.backbone}")
logger.info(f"device_name: {hparams.device_name}")
logger.info(f"gpus: {hparams.gpus}")
logger.info(f"n_workers: {hparams.n_workers}")
logger.info(f"image_size: {hparams.image_size}")
logger.info(f"seed: {hparams.seed}")
logger.info(f"min_epochs: {hparams.min_epochs}")
logger.info(f"max_epochs: {hparams.max_epochs}")
logger.info(f"patience: {hparams.patience}")
logger.info(f"train_batch_size: {hparams.train_batch_size}")
logger.info(f"val_batch_size: {hparams.val_batch_size}")
logger.info(f"n_splits: {hparams.n_splits}")
logger.info(f"test_size: {hparams.test_size}")
logger.info(f"learning rate: {hparams.lr}")
logger.info(f"weight_decay: {hparams.weight_decay}")
logger.info(f"epsilon: {hparams.epsilon}")
logger.info(f"amsgrad: {hparams.amsgrad}")
logger.info(f"betas: {hparams.betas}")
logger.info(f"precision: {hparams.precision}")
logger.info(f"gradient_clip_val: {hparams.gradient_clip_val}")
logger.info(f"eta_min: {hparams.eta_min}")
logger.info(f"log_dir: {hparams.log_dir}")
logger.info(f"log_name: {hparams.log_name}")

# Log any notes if they exist
if "log_notes" in locals():
    logger.info(f"Notes: {log_notes}")


# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
val_loss_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_loss", 
        patience=hparams.patience, 
        mode="min", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
            
    # Save val scores
    val_loss_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
val_loss_scores = [i.item() for i in val_loss_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [val_loss_scores],
            "mean_score": [np.mean(val_loss_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {val_loss_scores}")
logger.info("Training complete.")

[2022-12-04 17:38:04] 128199480.py[  11] : INFO  backbone: resnext101_32x4d
[2022-12-04 17:38:04] 128199480.py[  12] : INFO  device_name: NVIDIA GeForce RTX 3090
[2022-12-04 17:38:04] 128199480.py[  13] : INFO  gpus: [0]
[2022-12-04 17:38:04] 128199480.py[  14] : INFO  n_workers: 128
[2022-12-04 17:38:04] 128199480.py[  15] : INFO  image_size: [1024, 1024]
[2022-12-04 17:38:04] 128199480.py[  16] : INFO  seed: 14
[2022-12-04 17:38:04] 128199480.py[  17] : INFO  min_epochs: 75
[2022-12-04 17:38:04] 128199480.py[  18] : INFO  max_epochs: 100
[2022-12-04 17:38:04] 128199480.py[  19] : INFO  patience: 11
[2022-12-04 17:38:04] 128199480.py[  20] : INFO  train_batch_size: 4
[2022-12-04 17:38:04] 128199480.py[  21] : INFO  val_batch_size: 4
[2022-12-04 17:38:04] 128199480.py[  22] : INFO  n_splits: 3
[2022-12-04 17:38:04] 128199480.py[  23] : INFO  test_size: 0.1
[2022-12-04 17:38:04] 128199480.py[  24] : INFO  learning rate: 0.0001
[2022-12-04 17:38:04] 128199480.py[  25] : INFO  weight_deca

[2022-12-04 18:06:21] 2786500167.py[  95] : INFO  0-38 | lr : 0.000051 | val_loss : 1.4469 | val_roc_auc : 0.8223 | data_load_times : 45.54 | batch_run_times : 45.93
[2022-12-04 18:07:05] 2786500167.py[  95] : INFO  0-39 | lr : 0.000048 | val_loss : 1.4414 | val_roc_auc : 0.8340 | data_load_times : 46.12 | batch_run_times : 46.57
[2022-12-04 18:07:49] 2786500167.py[  95] : INFO  0-40 | lr : 0.000044 | val_loss : 1.4386 | val_roc_auc : 0.8383 | data_load_times : 47.41 | batch_run_times : 47.79
[2022-12-04 18:08:33] 2786500167.py[  95] : INFO  0-41 | lr : 0.000041 | val_loss : 1.3977 | val_roc_auc : 0.8416 | data_load_times : 46.18 | batch_run_times : 46.56
[2022-12-04 18:09:16] 2786500167.py[  95] : INFO  0-42 | lr : 0.000038 | val_loss : 1.4468 | val_roc_auc : 0.8127 | data_load_times : 48.46 | batch_run_times : 49.03
[2022-12-04 18:09:59] 2786500167.py[  95] : INFO  0-43 | lr : 0.000035 | val_loss : 1.3952 | val_roc_auc : 0.8327 | data_load_times : 45.22 | batch_run_times : 45.61
[202

[2022-12-04 18:36:45] 2786500167.py[  95] : INFO  1-5 | lr : 0.000076 | val_loss : 1.8982 | val_roc_auc : 0.7846 | data_load_times : 45.77 | batch_run_times : 46.15
[2022-12-04 18:37:29] 2786500167.py[  95] : INFO  1-6 | lr : 0.000067 | val_loss : 1.7872 | val_roc_auc : 0.8554 | data_load_times : 43.61 | batch_run_times : 44.00
[2022-12-04 18:38:13] 2786500167.py[  95] : INFO  1-7 | lr : 0.000057 | val_loss : 1.7390 | val_roc_auc : 0.8708 | data_load_times : 45.05 | batch_run_times : 45.45
[2022-12-04 18:38:58] 2786500167.py[  95] : INFO  1-8 | lr : 0.000048 | val_loss : 1.7378 | val_roc_auc : 0.8485 | data_load_times : 46.06 | batch_run_times : 46.43
[2022-12-04 18:39:43] 2786500167.py[  95] : INFO  1-9 | lr : 0.000038 | val_loss : 1.6998 | val_roc_auc : 0.8504 | data_load_times : 45.64 | batch_run_times : 46.02
[2022-12-04 18:40:28] 2786500167.py[  95] : INFO  1-10 | lr : 0.000029 | val_loss : 1.6963 | val_roc_auc : 0.8595 | data_load_times : 45.16 | batch_run_times : 45.62
[2022-12-

[2022-12-04 19:13:42] 2786500167.py[  95] : INFO  1-55 | lr : 0.000008 | val_loss : 1.3175 | val_roc_auc : 0.8533 | data_load_times : 45.93 | batch_run_times : 46.37
[2022-12-04 19:14:26] 2786500167.py[  95] : INFO  1-56 | lr : 0.000007 | val_loss : 1.3262 | val_roc_auc : 0.8594 | data_load_times : 37.67 | batch_run_times : 38.33
[2022-12-04 19:15:10] 2786500167.py[  95] : INFO  1-57 | lr : 0.000006 | val_loss : 1.3547 | val_roc_auc : 0.8594 | data_load_times : 46.26 | batch_run_times : 46.67
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-04 19:15:54] 2786500167.py[  95] : INFO  1-58 | lr : 0.000005 | val_loss : 1.3471 | val_roc_auc : 0.8554 | data_load_times : 44.94 | batch_run_times : 45.31
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-04 19:16:38] 2786500167.py[  95] : INFO  1-59 | lr : 0.000005 | val

[2022-12-04 19:39:41] 2786500167.py[  95] : INFO  2-15 | lr : 0.000100 | val_loss : 1.7644 | val_roc_auc : 0.7803 | data_load_times : 45.98 | batch_run_times : 46.35
[2022-12-04 19:40:26] 2786500167.py[  95] : INFO  2-16 | lr : 0.000100 | val_loss : 1.7522 | val_roc_auc : 0.7796 | data_load_times : 46.44 | batch_run_times : 46.83
[2022-12-04 19:41:11] 2786500167.py[  95] : INFO  2-17 | lr : 0.000100 | val_loss : 1.7187 | val_roc_auc : 0.7906 | data_load_times : 44.71 | batch_run_times : 45.07
[2022-12-04 19:41:57] 2786500167.py[  95] : INFO  2-18 | lr : 0.000099 | val_loss : 1.7417 | val_roc_auc : 0.7633 | data_load_times : 44.06 | batch_run_times : 44.45
[2022-12-04 19:42:41] 2786500167.py[  95] : INFO  2-19 | lr : 0.000098 | val_loss : 1.6833 | val_roc_auc : 0.8021 | data_load_times : 44.54 | batch_run_times : 44.93
[2022-12-04 19:43:26] 2786500167.py[  95] : INFO  2-20 | lr : 0.000097 | val_loss : 1.6373 | val_roc_auc : 0.8256 | data_load_times : 46.20 | batch_run_times : 46.72
[202

Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-04 20:16:26] 2786500167.py[  95] : INFO  2-64 | lr : 0.000100 | val_loss : 1.3614 | val_roc_auc : 0.8361 | data_load_times : 46.44 | batch_run_times : 46.88
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-04 20:17:11] 2786500167.py[  95] : INFO  2-65 | lr : 0.000100 | val_loss : 1.4332 | val_roc_auc : 0.8323 | data_load_times : 43.15 | batch_run_times : 43.55
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-04 20:17:55] 2786500167.py[  95] : INFO  2-66 | lr : 0.000100 | val_loss : 1.4311 | val_roc_auc : 0.8332 | data_load_times : 46.80 | batch_run_times : 47.21
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training

## Validation Inference

In [7]:
# Get model run path and define chosen fold
log_dir = "../logs/logs"
#model_run = "2022_11_08_14:57:52"
model_run = hparams.log_name
model_run_path = os.path.join(log_dir, model_run)
#best_fold = 1
best_fold = val_loss_scores.index(min(val_loss_scores))

# Get best model for chosen fold
model_run_dir = os.listdir(model_run_path)
model_folds = [i for i in model_run_dir if i.startswith(f"fold={best_fold}")]
model_folds_scores = [float(i.split("val_loss=")[1].split("-")[0]) for i in model_folds]
model_name = model_folds[model_folds_scores.index(min(model_folds_scores))]
model_path = os.path.join(model_run_path, model_name)

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(model_path)["state_dict"]
)
model.eval()

# Retrieve validation indices for chosen fold
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    if fold_i == best_fold:
        break

# Select fold validation images
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Create predictions looped by batch
counter = 0
val_i_batch = []
val_idx_batch = []
scores_df = pd.DataFrame()

for i, idx in tqdm(enumerate(val_index)):
    counter += 1
    val_i_batch.append(i) # arrays don't preserve index so need ordered index values
    val_idx_batch.append(idx) # for preserved index
    
    # Run inference for val_batch_size
    if counter == hparams.val_batch_size:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        # Reset counter and batch
        counter = 0
        val_i_batch = []
        val_idx_batch = []
        
    # Run inference for remaining batch
    elif idx == val_index[-1]:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

        
# Write predictions to log
scores_df.to_csv(
    os.path.join(model_run_path, f"{model_run}_preds_fold_{best_fold}.csv"),
    index=False
)

100it [07:51,  4.71s/it]


In [8]:
scores_df

,img_path,label,ant,bedbug,bee,horsefly,mite,mosquito,none,tick
0,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,0.157181,0.171592,0.101942,0.135047,0.142920,0.094712,0.106189,0.090416
1,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,0.156795,0.174067,0.101753,0.137763,0.142289,0.094485,0.104688,0.088160
2,../data/cleaned/none/49850884a00703afe5aab78c3...,none,0.155509,0.171901,0.102669,0.135296,0.143230,0.096477,0.104323,0.090595
3,../data/cleaned/none/8dab219e5fa4d9a05d42b6f3a...,none,0.155182,0.171211,0.102641,0.134951,0.142560,0.096005,0.106050,0.091399
4,../data/cleaned/none/d9c0f49c789a29fb55370319e...,none,0.155995,0.169098,0.102976,0.133203,0.142510,0.096468,0.107006,0.092743
...,...,...,...,...,...,...,...,...,...,...
95,../data/cleaned/ant/b7d95c4a5fe54f5bbfb3442ceb...,ant,0.156070,0.170976,0.102797,0.134621,0.143496,0.096506,0.104566,0.090967
96,../data/cleaned/ant/8fadf11644c8c4877b64bb61cb...,ant,0.156352,0.170685,0.102686,0.134540,0.143557,0.095951,0.104864,0.091366
97,../data/cleaned/ant/ab1b890b774b4cc6b71459a595...,ant,0.155295,0.168114,0.103171,0.132546,0.142086,0.096955,0.108024,0.093808
98,../data/cleaned/ant/d170bac72faf6447bd8d192744...,ant,0.157170,0.171674,0.102061,0.135350,0.142550,0.095383,0.106186,0.089625


## Validation Analysis

In [9]:
print(f"{len(scores_df['img_path'].unique())} unique image paths.")

100 unique image paths.


In [10]:
print("Validation label counts:")
print(scores_df["label"].value_counts())

Validation label counts:
bedbug      19
tick        18
mosquito    16
ant         16
none         9
horsefly     8
mite         7
bee          7
Name: label, dtype: int64


In [11]:
print("Validation prediction counts:")
print(
    pd.melt(
        scores_df,
        id_vars=["img_path", "label"],
        value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
        var_name="pred_label",
        value_name="pred_prob"
    ).sort_values(["img_path", "pred_prob"], ascending=False) \
    .groupby(["img_path", "label"]).first()["pred_label"] \
    .value_counts()
)

Validation prediction counts:
bedbug    100
Name: pred_label, dtype: int64


In [12]:
# Probability stats by label
pd.concat(
    [
        pd.DataFrame(scores_df.iloc[:, 2:].mean(), columns=["mean"]),
        pd.DataFrame(scores_df.iloc[:, 2:].std(), columns=["std"]),
        pd.DataFrame(scores_df.iloc[:, 2:].min(), columns=["min"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.25)),
        pd.DataFrame(scores_df.iloc[:, 2:].median(), columns=["median"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.75)),
        pd.DataFrame(scores_df.iloc[:, 2:].max(), columns=["max"]),
        pd.DataFrame(scores_df.iloc[:, 2:].max() - scores_df.iloc[:, 2:].min(), columns=["range"])
    ], 
    axis=1
)

,mean,std,min,0.25,median,0.75,max,range
ant,0.156080,0.000804,0.153455,0.155632,0.156085,0.156480,0.159257,0.005802
bedbug,0.171089,0.001355,0.168114,0.170134,0.170983,0.171685,0.176004,0.007891
bee,0.102579,0.000520,0.100200,0.102429,0.102658,0.102876,0.103554,0.003353
horsefly,0.134666,0.001175,0.132465,0.133895,0.134582,0.135249,0.138961,0.006496
mite,0.142823,0.000652,0.141142,0.142500,0.142850,0.143092,0.147121,0.005980
mosquito,0.096011,0.000966,0.090462,0.095679,0.096073,0.096584,0.097796,0.007334
none,0.105744,0.001070,0.102856,0.104945,0.105798,0.106372,0.108368,0.005512
tick,0.091008,0.001301,0.086169,0.090378,0.091209,0.091726,0.093808,0.007639


In [13]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).pivot_table(
    index=["label"],
    columns=["pred_label"],
    aggfunc="mean"
)

pred_prob                                                    \
pred_label       ant    bedbug       bee  horsefly      mite  mosquito   
label                                                                    
ant         0.156292  0.171452  0.102459  0.135144  0.143237  0.095804   
bedbug      0.156064  0.171579  0.102525  0.134983  0.142824  0.096090   
bee         0.156413  0.170648  0.102560  0.134425  0.142751  0.095857   
horsefly    0.155697  0.171144  0.102574  0.134640  0.142263  0.096003   
mite        0.155780  0.171025  0.102646  0.134429  0.142830  0.096069   
mosquito    0.156085  0.170422  0.102790  0.134131  0.142905  0.096321   
none        0.155751  0.171207  0.102620  0.134934  0.142591  0.095997   
tick        0.156224  0.170956  0.102517  0.134446  0.142773  0.095885   

                                
pred_label      none      tick  
label                           
ant         0.105142  0.090470  
bedbug      0.105357  0.090578  
bee         0.106070  0.091275  
horsefly    0.106448  0.091231  
mite        0.105871  0.091350  
mosquito    0.105822  0.091525  
none        0.105813  0.091087  
tick        0.106094  0.091105

In [14]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).sort_values(["img_path", "pred_prob"], ascending=False).groupby(["img_path", "label"]).first()

,,pred_label,pred_prob
img_path,label,,
../data/cleaned/ant/24aca08f4aeea83c077a9721c85b5a6ff8ef7d3d.jpg,ant,bedbug,0.170619
../data/cleaned/ant/26b8fb30b3ddd29f8e5763b8b9a63cda83006c7a.jpg,ant,bedbug,0.172482
../data/cleaned/ant/448b50c6778952be3df52a6a26affeff4844b7f4.jpg,ant,bedbug,0.171213
../data/cleaned/ant/451d80e25d8f18e8a68174ff8ff651c435bd4700.jpg,ant,bedbug,0.171935
../data/cleaned/ant/51782d78019c7b5be5333e7bc2dab3469f88fec7.jpg,ant,bedbug,0.171627
...,...,...,...
../data/cleaned/tick/c99e1bba324bc3b31bfab79473a50baeb0c87a2a.jpg,tick,bedbug,0.171142
../data/cleaned/tick/d842edcf4167941990cb0b05c8a3c58db001c8d6.jpg,tick,bedbug,0.170778
../data/cleaned/tick/ebeb916d8413d0b3387c84896fb99ddf6eb2fe13.jpg,tick,bedbug,0.174221
